<a href="https://colab.research.google.com/github/neuroboy-sohaib/ML-Deep-Learning-codes/blob/master/Text_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install a Drive FUSE wrapper.
 # https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null 
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 130942 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.6-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [0]:
# Generate auth tokens for Colab
from google.colab import auth 
auth.authenticate_user()

In [0]:
# Generate creds for the Drive FUSE library.

from oauth2client.client import GoogleCredentials 
creds = GoogleCredentials.get_application_default()
import getpass 
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass() 
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
layers = keras.layers
models = keras.models


# This code was tested with TensorFlow v1.8
print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 1.14.0


In [0]:
data = pd.read_csv("bbc-text.csv")

In [0]:
data.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [0]:
data['category'].value_counts()

business         122
sport            109
politics          92
entertainment     78
tech              70
Name: category, dtype: int64

In [0]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords                   #Stopwords corpus
from nltk.stem import PorterStemmer                 # Stemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
stop = set(stopwords.words('english')) 

In [0]:
import re
temp =[]
snow = nltk.stem.SnowballStemmer('english')
for sentence in data['text']:
    sentence = sentence.lower()                 # Converting to lowercase
    cleanr = re.compile('<.*?>')
    sentence = re.sub(cleanr, ' ', sentence)        #Removing HTML tags
    sentence = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    sentence = re.sub(r'[.|,|)|(|\|/]',r' ',sentence)        #Removing Punctuations
    
    words = [snow.stem(word) for word in sentence.split() if word not in stopwords.words('english')]   # Stemming and removing stopwords
    temp.append(words)
    
final_X = temp    

In [0]:
sent = []
for row in final_X:
    sequ = ''
    for word in row:
        sequ = sequ + ' ' + word
    sent.append(sequ)

final_X = sent
print(final_X[1])

 worldcom boss left book alon former worldcom boss berni ebber accus overse $11bn £5 8bn fraud never made account decis wit told juror david myer made comment question defenc lawyer argu mr ebber respons worldcom problem phone compani collaps 2002 prosecutor claim loss hidden protect firm share mr myer alreadi plead guilti fraud assist prosecutor monday defenc lawyer reid weingarten tri distanc client alleg cross examin ask mr myer ever knew mr ebber make account decis awar mr myer repli ever know mr ebber make account entri worldcom book mr weingarten press repli wit mr myer admit order fals account entri request former worldcom chief financi offic scott sullivan defenc lawyer tri paint mr sullivan admit fraud testifi later trial mastermind behind worldcom account hous card mr ebber team meanwhil look portray affabl boss admiss pe graduat economist whatev abil mr ebber transform worldcom relat unknown $160bn telecom giant investor darl late 1990s worldcom problem mount howev competit 

In [0]:
w2v_data = final_X
splitted = []
for row in w2v_data: 
    splitted.append([word for word in row.split()])     #splitting words

In [0]:
from gensim.models import Word2Vec   
train_w2v = Word2Vec(splitted,min_count=5,size=100, workers=4)

In [0]:
train_w2v.vocabulary()

In [0]:
# import the existing word and sentence tokenizing 
# libraries 
from nltk.tokenize import sent_tokenize, word_tokenize 
tokenised =[]	 
  
for i in final_X:  
  tokenised.append(word_tokenize(i))

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
X=[]
vectorizer = CountVectorizer()
for i in tokenised:
  X.append(vectorizer.fit_transform(i))


In [0]:
len(final_X)

471

In [0]:
import scipy
dense=[]
for i in X:
  dense.append(scipy.sparse.csr_matrix.todense(i))
print(dense)  

[matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]), matrix([[0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]), matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]), matrix([[0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]]), matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
  

In [0]:
import random
import numpy as np

"""Appends the minimal required amount of zeroes at the end of each 
array in the jagged array `M`, such that `M` looses its jagedness."""

maxlen = max(len(r) for r in dense)

for j in dense:
  for n in range(len(j), maxlen):
    print(c=c+1)
    np.append(j,0)

In [0]:
a=np.ones(3)+np.ones(3)

(a[1,])

2.0

In [0]:
g=np.zeros(1000)
# Create an array of zeros with the reference shape
# Create a list of slices from offset to offset + shape in each dimension
insertHere = [slice(offsets[dim], offsets[dim] + dense.shape[dim]) for dim in range(dense.ndim)]
# Insert the array in the result at the specified offsets
result[insertHere] = dense

In [0]:
result[:dense.shape[0],:dense.shape[1]] = dense

In [0]:
train_size = int(len(data) * .7)
def train_test_split(data, train_size):
    train = data[:train_size]
    test = data[train_size:]
    return train, test

In [0]:
train_cat, test_cat = train_test_split(data['category'], train_size)
train_text, test_text = train_test_split(data['text'], train_size)

In [0]:
max_words = 1000
tokenize = keras.preprocessing.text.Tokenizer(num_words=max_words, 
                                              char_level=False)

In [0]:
tokenize.fit_on_texts(train_text) # fit tokenizer to our training text data
x_train = tokenize.texts_to_matrix(train_text)
x_test = tokenize.texts_to_matrix(test_text)

In [0]:
len(dense[2])

130

In [0]:
# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(train_cat)
y_train = encoder.transform(train_cat)
y_test = encoder.transform(test_cat)

In [0]:
# Converts the labels to a one-hot representation
num_classes = np.max(y_train) + 1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# This model trains very quickly and 2 epochs are already more than enough
# Training for more epochs will likely lead to overfitting on this dataset
# You can try tweaking these hyperparamaters when using this model with your own data
batch_size = 32
epochs = 5
drop_ratio = 0.5

In [0]:
# Build the model
model = models.Sequential()
model.add(layers.Dense(512, input_shape=(max_words,)))
model.add(layers.Activation('relu'))
# model.add(layers.Dropout(drop_ratio))
model.add(layers.Dense(num_classes))
model.add(layers.Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
# model.fit trains the model
# The validation_split param tells Keras what % of our training data should be used in the validation set
# You can see the validation loss decreasing slowly when you run this
# Because val_loss is no longer decreasing we stop training to prevent overfitting
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.3)

Train on 1246 samples, validate on 534 samples
Epoch 1/5
1246/1246 [==============================] - 0s 285us/sample - loss: 0.5044 - acc: 0.8628 - val_loss: 0.1773 - val_acc: 0.9457
Epoch 2/5
1246/1246 [==============================] - 0s 183us/sample - loss: 0.0477 - acc: 0.9936 - val_loss: 0.1516 - val_acc: 0.9476
Epoch 3/5
1246/1246 [==============================] - 0s 175us/sample - loss: 0.0157 - acc: 1.0000 - val_loss: 0.1289 - val_acc: 0.9607
Epoch 4/5
1246/1246 [==============================] - 0s 204us/sample - loss: 0.0073 - acc: 1.0000 - val_loss: 0.1279 - val_acc: 0.9588
Epoch 5/5
1246/1246 [==============================] - 0s 191us/sample - loss: 0.0048 - acc: 1.0000 - val_loss: 0.1235 - val_acc: 0.9607


In [0]:
# Evaluate the accuracy of our trained model
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

445/445 [==============================] - 0s 91us/sample - loss: 0.1330 - acc: 0.9551
Test loss: 0.13303094901395648
Test accuracy: 0.9550562
